# Converting EEG data into BIDS Format
## Author: Will Decker

The following script converts raw BrainVision files into BIDS format using MNE. Information on BIDS, BIDS formating and other related documentation can be found [here](https://bids-specification.readthedocs.io/en/stable/). Information about MNE, its implementation and related documentation can be found [here](https://mne.tools/stable/overview/index.html).

### Load MNE and other necessary modules/packages

In [2]:
import os
import os.path as op
import mne
from mne_bids import write_raw_bids, BIDSPath, print_dir_tree
from mne_bids.stats import count_events

### Load in raw data

In [ ]:
# identify and display current raw data directory
rawdata = " "
print_dir_tree(rawdata) # expand/display file tree

# identify '.vhdr' files in raw data directory and load them into empty
raw_data = [] # empty list to append rawdata info to
for filename in os.listdir(rawdata):
    if filename.endswith(".vhdr"):
        file_path = os.path.join(rawdata, filename)
        raw = mne.io.read_raw_brainvision(file_path)
        raw_data.append(raw)

# combine raw data into a single raw object
raw = mne.concatenate_raws(raw_data)

# print information about raw data
print(raw.info)

### Montage

Option 1: If you DO NOT have access to original montage file

In [ ]:

# get list of standard montages
get_montage = mne.channels.get_builtin_montages(descriptions = True)
for montage_name, montage_description in get_montage: 
    print(f'{montage_name}: {montage_description}')
    montage = montage_name
    break

# append montage to raw object
standard_montage = mne.channels.make_standard_montage(montage)
raw.set_montage(standard_montage)

# display montage
raw.plot_sensors()

Option 2: If you DO have access to original montage file

In [ ]:
# path to montage file
montage_file = " "

# read in file
montage = mne.channels.read_custom_montage(montage_file)

# append montage to raw object
raw.set_montage(montage) # attribute montage to raw object

# dsiplay montage
raw.plot_sensors()

### Convert EEG data to BIDS format

In [ ]:
# identify necesary paths and  initialize variables to be used in BIDS files
bids_root = " " # path to new BIDS output
task = " " # task
session = " " # session

# subject IDs
subject_ID = [ ] # e.g. ['001', '002'...'034']

# or

# subject_ID = range(1,20) # Creates a vector, B,  of numbers, 1 through 20. Change this range to satsify participant ID requirements

# create BID compliant data based on subject ID numbers
for ids in subject_ID:
    try:
        bids_path = BIDSPath(subject=ids, task=task, root=bids_root)
        write_raw_bids(raw, bids_path, overwrite=True, allow_preload= True, format='BrainVision')
    except FileNotFoundError: # continue through loop
        print("Skipping file")
        continue

### Display overview of new BIDS dataset

In [ ]:
counts = count_events(bids_root)
counts

### References

In [ ]:
readme = op.join(bids_root, 'README')
with open(readme, 'r', encoding='utf-8-sig') as fid:
    text = fid.read()
print(text)

Pernet, C.R., Appelhoff, S., Gorgolewski, K.J. et al. EEG-BIDS, an extension to the brain imaging data structure for electroencephalography. Sci Data 6, 103 (2019). https://doi.org/10.1038/s41597-019-0104-8